In [4]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests

In [5]:
class HelperFunctions():
    #input: a list of quarters such as "Q2 2022"
    #iterates through a finds a date/year in the file
    def getYears(self,quarters): 
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    
    def getUniqueYears(self,quarters):
        return set(self.getYears(quarters))
    
    #input: compiledStatement
    #returns all quarters but removes first col of label
    def getQuarters(self, compiledStatement): 
        return list(compiledStatement.columns[1:])
    
    def getPossibleFilingsFromYears(self, years): 
        possible = [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"]]
        possible += [q + " " + str(year) for year in years for q in ["6mo","9mo"]]
        return possible
    
    #input: Excel dictionary with sheet name and the sheet dataframe
    #finds all possible filing names post fromDate and filters through dictionary keys to find corresponding filling
    def getExcelSheetsPostDate(self,Excel):  
        years = range(self.fromDate,self.endDate+1)
        names = self.getPossibleFilingsFromYears(years)
        if self.includeYearEnd:
            names = ["Year Ended "+str(self.fromDate-1)]+names
        files = [filing for name in names for filing in Excel.keys() if name in filing] 
        Excel = {file:Excel[file] for file in files}
        return Excel
    
    #input: compiledStatement
    #find all years in statement and get all possible filings in order then only keep the ones in actually in the statement
    def reorderQuarters(self,compiledStatement): 
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        cols = self.getPossibleFilingsFromYears(years)
        cols = [col for col in cols if col in quarters]
        compiledStatement = compiledStatement[["label"] + cols]
        return compiledStatement
    
    
    def cleanup_label(self, label):
        new_label = label.capitalize()
        new_label = re.sub("\([\w\W]+?\)","",new_label) #replace parenthesis wtih nothing
        new_label = re.sub(" \$|\s*\d+\,*", "", new_label) #replace money amounts in label with nothing
        new_label = new_label.replace("gain","loss").replace("decrease","increase").replace("  "," ").strip()
        return new_label
    
    
    #input: Excel dictionary with sheet name and the sheet dataframe
    #go through each possible sheet and merge into a compiledStatement
    #to do this I use an iterative approach where I start from a completely empty dataframe and then go through each row of compiledStatement and sheet based on index. 
    #if the label match, i add data from both. if the labels don't match then I will by default add the previously compiledStatement and continue moving
    def addData(self, Excel):
        compiledStatement = pd.DataFrame(columns=["label"])
        excel = list(Excel.items())
        excel.reverse()
        for quarter,sheet in excel:
            quarter =  re.sub("\s\([\w\W]+?\)","",quarter) #remove the parenthesis ie. (2021) from the quarter
            sheet.columns = ["label",quarter]
            
            sheet = sheet.dropna(subset="label", axis=0, how="any")
            sheet = sheet[(~sheet[quarter].astype("string").str.contains("[a-zA-Z]",regex=True))|sheet[quarter].isnull()] #remove rows where the numerical data is a string ie. "Hotel"
            sheet.label = sheet.label.apply(self.cleanup_label) #normalize the label
            
            sheeti = statementi = 0
            newStatement = pd.DataFrame()
            
            while sheeti < len(sheet) and statementi < len(compiledStatement):
            #merge the current compiledStatement and new sheet into one statement 
            #go through each row of the two statements via index; stop when reach bottom of one
            
                sheet_label = sheet.iloc[sheeti].label
                statement_label = compiledStatement.iloc[statementi].label
                newRowStatement = compiledStatement.iloc[statementi:statementi+1].reset_index(drop=True) #grab the row and drop the index so that they merge
                newRowSheet = sheet.iloc[sheeti:sheeti+1].reset_index(drop=True)

                #check if the labels are the same 
                if statement_label == sheet_label:
                    #if labels are the same then create a new row by concating the rows
                    newRowSheet = newRowSheet.drop(columns="label")
                    newRow = pd.concat([newRowStatement,newRowSheet],axis=1)
                    newStatement = pd.concat([newStatement, newRow], ignore_index=True)
                    
                    #move on to the next row
                    sheeti+=1
                    statementi+=1
                    
                else:
                    maxi = min(sheeti+5,len(sheet))
                    
                    #next, before I append the final statement label, I want to check if the statement label corresponds to the next label for the sheet
                    #if it corresponds, then I will move the sheet up instead
                    if statement_label in list(sheet.iloc[sheeti:maxi].label): 
                        newRowSheet["value"] = sheeti #since a new label is being added, then add a value to the row to keep track of its original index
                        newStatement = pd.concat([newStatement,newRowSheet]) #concat the sheet row to the new statement
                        sheeti += 1
                        continue
                    
                    #if the two labels don't match, default is to add the row of the final statment
                    newStatement = pd.concat([newStatement,newRowStatement])
                    statementi += 1
        
            #if either sheet hasn't finished, then add the end of the statement to the new one
            sheet["value"] = sheet.index
            newStatement = pd.concat([newStatement,sheet.iloc[sheeti:len(sheet)]])
            newStatement = pd.concat([newStatement,compiledStatement.iloc[statementi:len(compiledStatement)]])

            compiledStatement = newStatement
        
        compiledStatement = compiledStatement.sort_values(by="value").drop(columns="value") #value is record of the order of the label in the original sheet
        compiledStatement = compiledStatement.reset_index(drop=True)
        return compiledStatement
    

In [6]:
class CompileIncomeStatement(HelperFunctions):
    #input: start date, end date, ticker
    #this class grabs all of the income statements from excel, filters them according to date, then it compiles each individual sheet into one statement
    def __init__(self,ticker,fromDate, endDate=2022, includeYearEnd=True):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        self.includeYearEnd = includeYearEnd #include the year end statement from before the end date
        
        self.path = os.path.join("../input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Income Statements All-{self.ticker}.xlsx")
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.compiledStatement = self.performIncomeMath(self.compiledStatement)
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)
    
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    #input: compiledStatement
    #perform math to find q4 of the income statement which is the total year end minus q3
    def performIncomeMath(self,compiledStatement): 
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
         #replace the nan values with 0 so that the subtraction between an nan value does not result in nan
        c = compiledStatement.copy().replace(r'\s+', np.nan, regex=True).fillna(0).drop(columns="label")
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                compiledStatement["Q4 "+year] = c["Year Ended "+year] - c["Q3 "+year] - c["Q2 "+year] - c["Q1 "+year]
        
        compiledStatement = compiledStatement.replace(0,np.nan)
        return compiledStatement

            

In [7]:
class Income:
    #input ticker, start date
    #creates one massive compiled statement with all filings from the start date
    #then creates a bunch of income statements compiled by year where filings are "more" normalized placed side by side that can be manually concated
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("../input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Income Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    #input: start date
    #iterate through each possible year and create compiled statement for that fiscal year
    #i found that the statements tend to be more normalized within one year so this strategy can prevent out of control long compiled statements
    def createSeparateStatements(self): 
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileIncomeStatement(ticker=self.ticker, fromDate=year,endDate=year, includeYearEnd=False)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileIncomeStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [8]:
income = Income("AAPL",2017)
income.compiledStatement



,label,Year Ended 2016,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Year Ended 2017,Q1 2018,Q2 2018,Q3 2018,...,Q4 2020,Year Ended 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022
0,Income statement [abstract],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Net sales,215639.00,78351.00,52896.00,45408.00,52579.00,229234.00,88293.00,61137.00,53265.00,...,64698.00,274515.00,111439.00,89584.00,81434.00,83360.00,365817.00,123945.00,97278.00,82959
2,Cost of sales,131376.00,48175.00,32305.00,27920.00,32648.00,141048.00,54381.00,37715.00,32844.00,...,40009.00,169559.00,67111.00,51505.00,46179.00,48186.00,212981.00,69702.00,54719.00,47074
3,Gross margin,84263.00,30176.00,20591.00,17488.00,19931.00,88186.00,33912.00,23422.00,20421.00,...,24689.00,104956.00,44328.00,38079.00,35255.00,35174.00,152836.00,54243.00,42559.00,35885
4,Operating expenses:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
5,Research and development,10045.00,2871.00,2776.00,2937.00,2997.00,11581.00,3407.00,3378.00,3701.00,...,4978.00,18752.00,5163.00,5262.00,5717.00,5772.00,21914.00,6306.00,6387.00,6797
6,"Selling, general and administrative",14194.00,3946.00,3718.00,3783.00,3814.00,15261.00,4231.00,4150.00,4108.00,...,4936.00,19916.00,5631.00,5314.00,5412.00,5616.00,21973.00,6449.00,6193.00,6012
7,Total operating expenses,24239.00,6817.00,6494.00,6720.00,6811.00,26842.00,7638.00,7528.00,7809.00,...,9914.00,38668.00,10794.00,10576.00,11129.00,11388.00,43887.00,12755.00,12580.00,12809
8,Operating income,60024.00,23359.00,14097.00,10768.00,13120.00,61344.00,26274.00,15894.00,12612.00,...,14775.00,66288.00,33534.00,27503.00,24126.00,23786.00,108949.00,41488.00,29979.00,23076
9,"Other income/, net",1348.00,821.00,587.00,540.00,797.00,2745.00,756.00,274.00,672.00,...,126.00,803.00,45.00,508.00,243.00,-538.00,258.00,-247.00,160.00,-10


In [571]:
class CompileCashflowStatement(HelperFunctions):
    def __init__(self,ticker,fromDate, endDate=2022, includeYearEnd=True):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        self.includeYearEnd = includeYearEnd 
        
        self.path = os.path.join("../input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Cashflow Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Cashflow Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.compiledStatement = self.performCashflowMath(self.compiledStatement)
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)

        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    #input: compiledStatement
    #perform math to find q2,q3,q4 of the income statement because filings only contain data for past 9mo and past 6mo
    def performCashflowMath(self, compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
        #Q2 and Q3 are deceptively 6mo and 9mo 
        quarters = [quarter.replace("Q2","6mo").replace("Q3","9mo") for quarter in quarters]
        compiledStatement.columns = ["label"] + quarters
        
        #find labels that contain "cash at beginning of period" and "cash at end of period" 
        #the cash at beginning of period is not the different in cash between (for ex) 6mo and 9mo 
        begin_period = [label for label in compiledStatement.label if "begin" in label and "cash" in label]
        end_period = [label for label in compiledStatement.label if "end" in label and "cash" in label]
        compiledStatement.index = compiledStatement.label
        
        #replace the nan values with 0 so that the subtraction between an nan value does not result in nan
        c = compiledStatement.copy().replace(r'\s+', np.nan, regex=True).fillna(0).drop(columns="label")
        
        for year in years:
            year = str(year)
            if "6mo "+year in quarters and "Q1 "+year in quarters: 
                # find Q2 
                compiledStatement["Q2 "+year] = c["6mo "+year] - c["Q1 "+year]
                # for ex. cash at beginning of period at Q2 is the same as cash at end of period for Q1
                compiledStatement.loc[begin_period,"Q2 "+year] = list(compiledStatement.loc[end_period,"Q1 "+year])
                compiledStatement.loc[end_period,"Q2 "+year] = list(compiledStatement.loc[end_period,"6mo "+year])
            if "9mo "+year in quarters and "6mo "+year in quarters: 
                #find Q3 
                compiledStatement["Q3 "+year] = c["9mo "+year] - c["6mo "+year]
                compiledStatement.loc[begin_period,"Q3 "+year] = list(compiledStatement.loc[end_period, "6mo "+year])
                compiledStatement.loc[end_period,"Q3 "+year] = list(compiledStatement.loc[end_period,"9mo "+year])

            if "Year Ended "+year in quarters and "9mo "+year in quarters: 
                #find Q3 
                compiledStatement["Q4 "+year] = c["Year Ended "+year] - c["9mo "+year]
                compiledStatement.loc[begin_period,"Q4 "+year] = list(compiledStatement["9mo "+year].loc[end_period])
                compiledStatement.loc[end_period,"Q4 "+year] = list(compiledStatement.loc[end_period,"Year Ended "+year])
        
#         #remove 6mo and 9mo columns
#         cols = [col for col in compiledStatement.columns if "9mo" not in col and "6mo" not in col]
#         compiledStatement = compiledStatement[cols] 
        
        compiledStatement = compiledStatement.reset_index(drop=True)
        compiledStatement = compiledStatement.replace(0,np.nan)
        return compiledStatement
    
   
            

In [581]:
pd.set_option('display.max_rows', 50)

CompileCashflowStatement("AAPL",2017).compiledStatement


,label,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Year Ended 2017,Q1 2018,Q2 2018,Q3 2018,Q4 2018,...,6mo 2018,9mo 2018,6mo 2019,9mo 2019,6mo 2020,9mo 2020,6mo 2021,9mo 2021,6mo 2022,9mo 2022
0,statement of cash flows [abstract],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,"cash, cash equivalents and restricted cash, be...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,25913.0,25913.0,50224.0,50224.0,39789.0,39789.0,35929.0,35929
2,"cash and cash equivalents, beginning balances",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"cash and cash equivalents, beginning of the year",NaN,NaN,NaN,NaN,20484.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"cash and cash equivalents, beginning of the pe...",20484.0,16371.0,15157.0,18571.0,NaN,20289.0,27491.0,45059.0,31971.0,...,20289.0,20289.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,supplemental cash flow disclosure:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
57,"cash paid for income taxes, net",3510.0,3368.0,2874.0,1839.0,11591.0,3551.0,2789.0,2479.0,1598.0,...,6340.0,8819.0,9497.0,11795.0,7505.0,8410.0,10276.0,18536.0,9301.0,12251
58,"cash and cash equivalents, end of the year",NaN,NaN,NaN,20289.0,20289.0,NaN,NaN,NaN,25913.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,"cash and cash equivalents, end of the period",16371.0,15157.0,18571.0,NaN,NaN,27491.0,45059.0,31971.0,NaN,...,45059.0,31971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [573]:
class Cashflow:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("../input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Cashflow Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    def createSeparateStatements(self):
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileCashflowStatement(ticker=self.ticker, fromDate=year,endDate=year, includeYearEnd=False)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileCashflowStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [582]:
cashflow = Cashflow("AAPL",2017)




In [583]:
cashflow.compiledStatement

,label,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Year Ended 2017,Q1 2018,Q2 2018,Q3 2018,Q4 2018,...,6mo 2018,9mo 2018,6mo 2019,9mo 2019,6mo 2020,9mo 2020,6mo 2021,9mo 2021,6mo 2022,9mo 2022
0,statement of cash flows [abstract],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,"cash, cash equivalents and restricted cash, be...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,25913.0,25913.0,50224.0,50224.0,39789.0,39789.0,35929.0,35929
2,"cash and cash equivalents, beginning balances",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"cash and cash equivalents, beginning of the year",NaN,NaN,NaN,NaN,20484.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"cash and cash equivalents, beginning of the pe...",20484.0,16371.0,15157.0,18571.0,NaN,20289.0,27491.0,45059.0,31971.0,...,20289.0,20289.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,supplemental cash flow disclosure:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
57,"cash paid for income taxes, net",3510.0,3368.0,2874.0,1839.0,11591.0,3551.0,2789.0,2479.0,1598.0,...,6340.0,8819.0,9497.0,11795.0,7505.0,8410.0,10276.0,18536.0,9301.0,12251
58,"cash and cash equivalents, end of the year",NaN,NaN,NaN,20289.0,20289.0,NaN,NaN,NaN,25913.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,"cash and cash equivalents, end of the period",16371.0,15157.0,18571.0,NaN,NaN,27491.0,45059.0,31971.0,NaN,...,45059.0,31971.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [578]:
class CompileBalanceStatement(HelperFunctions):
     def __init__(self,ticker,fromDate, endDate=2022, includeYearEnd=True):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        self.includeYearEnd = includeYearEnd 
        
        self.path = os.path.join("../input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Balance Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Balance Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.compiledStatement = self.performBalanceMath(self.compiledStatement)
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)

        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    
    def performBalanceMath(self, compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                compiledStatement["Q4 "+year] = compiledStatement["Year Ended "+year]
        return compiledStatement
    
   
            

In [579]:
class Balance:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("../input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Balance Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    def createSeparateStatements(self):
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileBalanceStatement(ticker=self.ticker, fromDate=year,endDate=year, includeYearEnd=False)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileBalanceStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [587]:
balance = Balance("AAPL",2017)
balance.compiledStatement



,label,Q1 2017,Q2 2017,Q3 2017,Q4 2017,Year Ended 2017,Q1 2018,Q2 2018,Q3 2018,Q4 2018,...,Q4 2020,Year Ended 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022
0,current assets:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,cash and cash equivalents,16371,15157,18571,20289,20289,27491,45059,31971,25913,...,38016,38016,36010,38466,34050,34940,34940,37119,28098,27502
2,marketable securities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40388,...,52927,52927,40816,31368,27646,27699,27699,26794,23413,20729
3,short-term marketable securities,44081,51944,58188,53892,53892,49662,42881,38999,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"accounts receivable, net",NaN,NaN,NaN,NaN,NaN,NaN,NaN,14104,23186,...,16120,16120,27101,18503,17475,26278,26278,30213,20815,21803
5,"accounts receivable, less allowances of and re...",NaN,NaN,12399,17874,17874,23440,14324,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"accounts receivable, less an allowance of at e...",NaN,11579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"accounts receivable, less an allowance of in e...",14057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,inventories,2712,2910,3146,4855,4855,4421,7662,5936,3956,...,4061,4061,4973,5219,5178,6580,6580,5876,5460,5433
9,vendor non-trade receivables,13920,9033,10233,17799,17799,27459,8084,12263,25809,...,21325,21325,31519,14533,16433,25228,25228,35040,24585,20439
